In [23]:
from bs4 import BeautifulSoup

In [24]:
import urllib.request

In [25]:
r = urllib.request.urlopen('http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts').read()

In [26]:
soup = BeautifulSoup(r)

/Users/nicklheureux/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [27]:
print type(soup)

SyntaxError: invalid syntax (<ipython-input-27-67d65311189a>, line 1)

In [ ]:
print soup.prettify()[0:1000]

I can use the heirarchical nature of HTML structure to grab precisely the content that I am interested in. I will grab all of the elements that are within div tags and are also members of class "ec_statements."


In [ ]:
from IPython.display import Image
Image('http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png')

The HTML tags contained in the angled brackets provide structural information (and sometimes formatting), which we probably don't care about in and of itself but is useful for selecting only the content relevant to our needs.
Beautiful Soup is essentially a set of wrapper functions that make it simple to select common HTML elements.

In [ ]:
Image('http://www.cs.toronto.edu/~shiva/cscb07/img/dom/treeStructure.png')

In [28]:
from IPython.display import HTML
HTML('<iframe src=http://www.aflcio.org/Legislation-and-Politics/Legislative-Alerts width=700 height=500></iframe>')


I can use the heirarchical nature of HTML structure to grab precisely the content that I am interested in. I will grab all of the elements that are within div tags and are also members of class "ec_statements."

In [30]:
letters = soup.find_all("div", class_="ec_statements")


This returns a collection of tag objects. This is not one of the normal python collections covered in the other tutorials, it is an object specific to the Beautiful Soup library. It can be iterated over, but most other standard methods won't work on it. We'll have to do some preprocessing to get out the content that we want.

In [31]:
print(type(letters))


<class 'bs4.element.ResultSet'>


By examining one of the elements in this collection, we can see that the information we want inside these objects, but we'll need to use more of Beautiful Soup's functionality and know something about HTML strucure to access them.


In [32]:
letters[0]


<div class="ec_statements">
<div id="legalert_title"><a href="/Legislation-and-Politics/Legislative-Alerts/Letter-to-Representatives-in-opposition-to-the-Local-Radio-Freedom-Act">Letter to Representatives in opposition to the Local Radio Freedom Act</a></div>
<div id="legalert_date">February 19, 2016</div>
</div>

Recall that we want 3 things: the text of the item as it appears on the website, the URL that is linked to (so we can scrape and analyze it later), and the date the letter was sent.

First we should consider how we are going to store this data. Since we want to maintain the association between the 3 things from each observation. A natural way to store this is as a nested dict. Dict keys must be unique, and some of our items have the same associated date, so we'll have to use one of the other items. We'll use the name.

Visible text is always placed between tags. On the rendered page, the name we want is an active link, but we can get just the associated text by using the get_text method on the a tags. You can also use contents, which returns a list instead of a string.

We'll go through all of the items in our letters collection, and for each one, pull out the name and make it a key in our dict. The value will be another dict, but we haven't yet found the contents for the other items yet so we'll just create assign an empty dict object.



In [35]:
lobbying = {}
for element in letters:
    lobbying[element.a.get_text()] = {}


Now we want to add to our inner dictionary by the linked URL.

In HTML, links are always enclosed in a tags as the href attribute. We can use this fact to easily pull out the links. But if we look at the URLs, we'll find that they are incomplete. Like many websites, this site uses relative paths.

In [36]:
letters[0].a["href"]


'/Legislation-and-Politics/Legislative-Alerts/Letter-to-Representatives-in-opposition-to-the-Local-Radio-Freedom-Act'

If we want to visit these links later we'll need the complete path, so we will add in a prefix to complete these paths.

In [ ]:
prefix = "www.aflcio.org"


Now we'll go through our letters collection again, and add a link key to our lobbying dictionaries.

In [ ]:
for element in letters:
    lobbying[element.a.get_text()]["link"] = prefix + element.a["href"]


Now we'll do the same thing with the date. Note that all of the dates are found within div tags with id "legalert_date". We can use this structure to grab the date information just as we did the link text. We know there is only one of these per item so we can use find instead of find_all.

In [ ]:
letters[0].find(id="legalert_date")


Now we'll add it into our lobbying dictionary. We can see above that we only want the text between the tags, so we'll pull that out before adding it into the dictionary.

In [ ]:
for element in letters:
    date = element.find(id="legalert_date").get_text()
    lobbying[element.a.get_text()]["date"] = date


In [ ]:
for item in lobbying.keys():
    print item + ": " + "\n\t" + "link: " + lobbying[item]["link"] + "\n\t" + "date: " + lobbying[item]["date"] + "\n\n" 


In [ ]:
import webbrowser
webbrowser.open_new("https://www.google.com/?gws_rd=ssl")

In [ ]:

print soup.prettify()